In [ ]:
import sys
sys.path.insert(1, '/home/linus/projects/fdl/crisp')

In [2]:
%pwd

'/Users/linus/projects/fdl/crisp/notebooks'

In [3]:
import os
import pandas as pd
os.chdir("..")

In [4]:
from models.Deconfounder import *
from models.NonLinearInvariantCausalPrediction import NonLinearInvariantCausalPrediction
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [5]:
from synthetic.facebook_synthetic_data_generator import generator_example

n_example = 2
dim_inv=2
n_bin=1
dim_spu=2
n_exp=2000 #int(2e2)
n_env=2
save_dir= 'data/synthetic'
test=False 



In [6]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        #'dataset_fp': '../data/test_multiclass.pkl',
        'dataset_fp' : None,
        'output_data_regime' : ["real-valued", "binary", "binary", "None", "multi-class", "real-valued"],
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
 #        'predictors': ['All'],
        'predictors': None,
        'environments': ['env_split'],
        'exclude': ['Subj_ID']
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [7]:
NLICP_options = config.get("ensemble_options").get('NLICP', {})
NLICP_args = {
            "max_set_size": 2,
            "alpha": 0.05,
            "seed": 12,
            "verbose": 1,
            "method": "MLP",
            "hidden_dim": 256
        }
NLICP_args.update(NLICP_options)

In [8]:
results = {}

for n_example in [1,2,3,5,6]:
    print("\n\nn_example:", n_example, "\n\n")
    generator_example(n_example, dim_inv, dim_spu, n_exp, n_env ,save_dir, test, n_bin)
    outfile = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test))
    if n_example == 6:
        outfile = os.path.join(save_dir,"data_fb_example_%s_dim_inv_%s_dim_spu_%s_n_exp_%s_n_env_%s_test_%s_n_bin_%s.pickle" %(n_example, dim_inv, dim_spu, n_exp, n_env,test,n_bin))
    df_train = pd.read_pickle(outfile)
    df_train = df_train[df_train["env_split"]==0]
    
    #df_train = df_train.drop(columns=["env_split", "Subj_ID"])
    #df_train.head()
    config["data_options"]["dataset_fp"] = outfile
    config["data_options"]["predictors"] = list(df_train.columns[0:dim_inv+dim_spu])
    environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

    NLICP_args["target"] = config["data_options"]["targets"]
    NLICP_args["output_data_regime"] = config["data_options"]["output_data_regime"][n_example-1]
    NLICP_args["columns"] = test_dataset.predictor_columns

    nlicp = NonLinearInvariantCausalPrediction(environment_datasets, val_dataset, test_dataset, NLICP_args)
    if not hasattr(nlicp, 'test_logits'): # nlicp.test() was not run after training bc no sets accepted -> nlicp.results() won't work
        nlicp_results_dict = {"result" : "no-accepted-sets"}
    else:
        nlicp_results_dict = nlicp.results()
    
    results[n_example] = nlicp_results_dict



n_example: 1 


Environments variables: {'E0': 0.1, 'E1': 1.5}
Generated Synthetic Data according to the Facebook setup Example: 1
Running a per sample experiment
Loaded  2  train environments
Env  0  has  808  samples
X shape  (808, 4)  y shape  (808, 1)
Env  1  has  812  samples
X shape  (812, 4)  y shape  (812, 1)
Loaded val set, X shape: (180, 4)  y shape:  (180, 1)
Loaded test set, X shape: (200, 4)  y shape:  (200, 1)
no accepted sets found for nonlinear ICP


n_example: 2 


Environments variables: {'E0': {'p': 0.95, 's': 0.3}, 'E1': {'p': 0.97, 's': 0.5}}
Generated Synthetic Data according to the Facebook setup Example: 2
Running a per sample experiment
Loaded  2  train environments
Env  0  has  826  samples
X shape  (826, 4)  y shape  (826, 1)
Env  1  has  794  samples
X shape  (794, 4)  y shape  (794, 1)
Loaded val set, X shape: (180, 4)  y shape:  (180, 1)
Loaded test set, X shape: (200, 4)  y shape:  (200, 1)
no accepted sets found for nonlinear ICP


n_example: 3 


Envi

In [ ]:
df_train.Target.unique()

In [ ]:
results

In [ ]:
#from synthetic.synthetic_generator import synthetic_generator
#df = synthetic_generator()
#df["Target"] = np.random.randint(0,3,size=df.shape[0])
#df.to_pickle("../data/test_multiclass.pkl")

In [ ]:
import pandas as pd
import pickle
#with open("../data/test.pkl",'rb') as file:
with open("../data/test_multiclass.pkl",'rb') as file:
    df = pickle.load(file)

In [ ]:
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

In [ ]:
# run DCF
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}
dcf_args["columns"] = test_dataset.predictor_columns

dcf = TorchMultiClassDeconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
print(dcf_results_dict)

In [ ]:
dcf_results_dict["results"]["Features"]

In [ ]:
dcf_results_dict["results"]["Features"]

In [ ]:
# run DCF
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": False # we cannot output pvalues in torch on the current build 
}
dcf_args["columns"] = test_dataset.predictor_columns

dcf = Deconfounder(environment_datasets, val_dataset, test_dataset, dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
print(dcf_results_dict)

In [ ]:
torch_ICP_results_dict['selected_features']